# Wheatley

> Next-generation scheduling problem solver based on GNNs and Reinforcement Learning

The authors of Wheatley created a [script for training](jssp/train.py) and a [script for solving](jssp/solve.py) the problem. Documentation is in [USAGE.md](docs/USAGE.md) For this quick showcase, I trained a model on random 15x15 instances (15 jobs, 15 machines). To solve benchmark instances, I just need to import the function implemented by Wheatley's authors. The model takes quite some time to train, and therefore the model in this showcase is trained very badly due to time constraints. Makespans will be therefore incredibly bad.

In [1]:
from pathlib import Path

import numpy as np
import torch

from generic.utils import decode_mask
from jssp.description import Description
from jssp.env.env import Env
from jssp.models.agent import Agent
from jssp.solution import Solution

# import Wheatley jssp solver
from jssp.solve import solve_instance
from jssp.utils.loaders import load_problem
from jssp.models.agent import Agent

# load trained agent
agent = Agent.load("saved_networks/agent.pkl")

# Benchmarks Wheatley can solve right now

Wheatley can solve only instances, which are the same size or smaller than those he has been trained on. Therefore I can 
I will now run Wheatley on all available JSSP benchmarks I currently have, which are 15x15 or smaller.

In [2]:
import os

def get_all_instances_in_taillard_specification():
    '''Lists all instances in Taillard specification'''
    matching_files = []
    root_dir = "../../../benchmarks/jssp/"
    target_string = "Taillard_specification"

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            if target_string in filepath:
                matching_files.append(filepath)

    return matching_files

In [3]:
for instance in sorted(get_all_instances_in_taillard_specification()):
    n_j, n_m, affectations, durations = load_problem(
        instance,
        taillard_offset=True,
        deterministic=True
    )

    if agent.env_specification.max_n_jobs < n_j:
        continue

    if agent.env_specification.max_n_machines < n_m:
        continue
        
    solution = solve_instance(
        agent, affectations, durations, True
    )
    print(f"Instance: {instance.split('/')[-1]}, jobs: {n_j}, machines: {n_m}, makespan: {solution.get_makespan()}")

Instance: abz5.txt, jobs: 10, machines: 10, makespan: 3749.0


/Users/marosbratko/Graph-neural-networks-and-deep-reinforcement-learning-in-job-scheduling/models/Wheatley/repo/jssp/env/observation.py:127: RuntimeWarning: invalid value encountered in cast
  "edge_index": edge_index.astype("int64"),


Instance: abz6.txt, jobs: 10, machines: 10, makespan: 2700.0
Instance: ft06.txt, jobs: 6, machines: 6, makespan: 80.0
Instance: ft10.txt, jobs: 10, machines: 10, makespan: 2768.0
Instance: la01.txt, jobs: 10, machines: 5, makespan: 1121.0
Instance: la02.txt, jobs: 10, machines: 5, makespan: 1171.0
Instance: la03.txt, jobs: 10, machines: 5, makespan: 1096.0
Instance: la04.txt, jobs: 10, machines: 5, makespan: 1296.0
Instance: la05.txt, jobs: 10, machines: 5, makespan: 839.0
Instance: la06.txt, jobs: 15, machines: 5, makespan: 1816.0
Instance: la07.txt, jobs: 15, machines: 5, makespan: 1801.0
Instance: la08.txt, jobs: 15, machines: 5, makespan: 2108.0
Instance: la09.txt, jobs: 15, machines: 5, makespan: 2001.0
Instance: la10.txt, jobs: 15, machines: 5, makespan: 2158.0
Instance: la16.txt, jobs: 10, machines: 10, makespan: 2673.0
Instance: la17.txt, jobs: 10, machines: 10, makespan: 2382.0
Instance: la18.txt, jobs: 10, machines: 10, makespan: 2602.0
Instance: la19.txt, jobs: 10, machines:

# Dynamic JSSP

In dynamic JSSP, only a subset of jobs is known at the beginning. The rest of jobs arrives dynamically online.

The following attempt to expand L2D to being dynamic is inspired by paper [Large-scale Dynamic Scheduling for Flexible Job-shop with Random Arrivals of New Jobs by Hierarchical Reinforcement Learning](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10114974), where authors schedule newly incoming jobs and reschedule not yet executed operations, already executed operations can not be rescheduled. During each rescheduling, they formulate static FJSP and solve it. They use cache for incoming jobs and an agent choosing either to add jobs from cache to scheduling problem, or keep them in cache. I will skip this agent and always add new jobs to scheduling problem.

Similarly to the paper, I will model the arrival of new jobs as poisson process with average arrival time following an exponential distribution.

In [4]:
from datetime import datetime

def parse_instance_taillard(filename):
    '''Parses instance written in Taillard specification: http://jobshop.jjvh.nl/explanation.php
    
      Args:
        filename - file containing the instance in Taillard specification

      Returns:
        number of jobs,
        number of machines,
        the processor times for each operation,
        the order for visiting the machines
    '''

    with open(filename, 'r') as f:
        # parse number of jobs J and machines M
        J, M = map(int, f.readline().split())

        # Initialize two empty numpy arrays with dimensions J x M
        processor_times = np.empty((J, M), dtype=int)
        orders_of_machines = np.empty((J, M), dtype=int)
    
        # Read the next J lines containing processor times
        for i in range(J):
            processor_times[i] = list(map(int, f.readline().split()))
    
        # Read the next J lines containing orders of machines
        for i in range(J):
            orders_of_machines[i] = list(map(int, f.readline().split()))

        return J, M, processor_times, orders_of_machines

def get_dynamics_jssp(instance):
    '''Turns static JSSP instance to dynamic

      Args:
        filename of static JSSP instance

      Returns:
        list of jobs known at the beginning
        dictionary of arriving jobs as  as {time_of_arrival: (operations, machines)} 
    '''
    J, M, processor_times, orders_of_machines = parse_instance_taillard(instance)

    indices = np.arange(J)
    np.random.shuffle(indices)

    # separate jobs into known jobs and arriving jobs
    jobs_known_at_the_beginning = [(processor_times[i], orders_of_machines[i]) for i in indices[J//2:]]
    arriving_jobs_indeces = indices[:J//2]

    # calculate beta = 1/lambda
    average_time_between_arrivals = (processor_times.mean() * len(processor_times[0])) / M
    
    t = 1
    arriving_jobs = {}
    for index in arriving_jobs_indeces:
        t += int(np.random.exponential(scale=average_time_between_arrivals))
        arriving_jobs[t] = (processor_times[index], orders_of_machines[index])

    return jobs_known_at_the_beginning, arriving_jobs

def save_static_jssp_taillard(jobs):
    '''Saves list of jobs as static JSSP instance in taillards specification
        
      Args:
        list of jobs to save

      Returns:
        filename where JSSP instance was saved to
    '''
    J, M = len(jobs), len(jobs[0][0])
    formatted_datetime = datetime.now().strftime("%Y%m%d%H%M%S")
    with open(f"/tmp/{J}_{M}_{formatted_datetime}.txt", 'w') as f:
        f.write(f"{J} {M}\n")
        for job in jobs:
            times, _ = job
            f.write(" ".join(map(str, times)) + '\n')
        for job in jobs:
            _, orders = job
            f.write(" ".join(map(str, orders)) + '\n')  

    return f"/tmp/{J}_{M}_{formatted_datetime}.txt"

In [5]:
# Dynamic jssp

def solve_instance_extended(
    agent: Agent,
    affectations: np.ndarray,
    durations: np.ndarray,
    deterministic: bool,
    partial_plan: list | None = None
) -> Solution:
    problem_description = Description(
        transition_model_config="simple",
        reward_model_config="Sparse",
        deterministic=deterministic,
        fixed=True,
        seed=0,
        affectations=affectations,
        durations=durations,
    )
    env_specification = agent.env_specification
    env = Env(problem_description, env_specification)

    done = False
    obs, info = env.reset(soft=True)
    actions = []

    i = 0
    while not done:
        if partial_plan is not None and i < len(partial_plan):
            action = partial_plan[i]
        else:
            action_masks = decode_mask(info["mask"])
            obs = agent.obs_as_tensor_add_batch_dim(obs)
            action = agent.predict(obs, deterministic=True, action_masks=action_masks)

        i += 1
        actions.append(action)
        obs, reward, done, _, info = env.step(action.long().item())
        solution = env.get_solution()

    return solution, solution.schedule, actions

In [6]:
def solve_dynamic_jssp(instance, agent):
    '''Turns static JSSP in Taillard specification instance to dynamic and solves it

      Args: 
        instance to solve

      Returns: 
        makespan
    '''
    # turn static JSSP instance to dynamic
    known_jobs, arriving_jobs = get_dynamics_jssp(instance)
    print(f"instance={instance.split('/')[-1]}, known_jobs={len(known_jobs)}, arriving_jobs={len(arriving_jobs)}")
    latest_time_of_arrival = max(arriving_jobs)
    print(f"Latest job arrives at {latest_time_of_arrival} and has total makespan {arriving_jobs[latest_time_of_arrival][0].max()}")
    
    # solve static JSSP with jobs known initially
    instance_partial = save_static_jssp_taillard(known_jobs)
    n_j, n_m, affectations, durations = load_problem(
        instance_partial,
        taillard_offset=True,
        deterministic=True
    )
    solution, schedule, actions = solve_instance_extended(
        agent, affectations, durations, True
    )

    # iterate over time
    t = 0
    while True:
        t += 1
        
        # no jobs left
        if not arriving_jobs:
            break
    
        # no job arrived
        if not t in arriving_jobs:
            continue
    
        # new job arrived, remove not yet executed operations from the plan
        partial_plan = []
        J, M = len(known_jobs), len(known_jobs[0][0])
        for action in actions:
            action = action.long().item()
            row = action // M
            col = action % M
    
            if schedule[row][col] < t:
                partial_plan.append(action)

        # add new job to the plan, with times shifted to current time t
        new_job = arriving_jobs.pop(t)
        known_jobs.append(new_job)
        instance_partial = save_static_jssp_taillard(known_jobs)
        n_j, n_m, affectations, durations = load_problem(
            instance_partial,
            taillard_offset=True,
            deterministic=True
        )
        
        # create new schedule WHILE REUSING THE ALREADY EXECUTED PLAN
        solution, schedule, actions = solve_instance_extended(
            agent, affectations, durations, True, partial_plan=actions
        )

    makespan = solution.get_makespan()
    return makespan


# experiment
BENCHMARKS_PATH = "../../../benchmarks/jssp/ft_instances/Taillard_specification/"
INSTANCES = [
    'ft06.txt'
]

for instance in INSTANCES:
    makespan = solve_dynamic_jssp(BENCHMARKS_PATH + instance, agent)
    print(f"Makespan of dynamic instance '{instance}': {makespan}")

    n_j, n_m, affectations, durations = load_problem(
        BENCHMARKS_PATH + instance,
        taillard_offset=True,
        deterministic=True
    )

    if agent.env_specification.max_n_jobs < n_j:
        continue

    if agent.env_specification.max_n_machines < n_m:
        continue
        
    solution = solve_instance(
        agent, affectations, durations, True
    )
    print(f"Makespan of static instance: {instance.split('/')[-1]}, jobs: {n_j}, machines: {n_m}, makespan: {solution.get_makespan()}")

instance=ft06.txt, known_jobs=3, arriving_jobs=3
Latest job arrives at 38 and has total makespan 10
Makespan of dynamic instance 'ft06.txt': 108.0
Makespan of static instance: ft06.txt, jobs: 6, machines: 6, makespan: 80.0
